In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
from astropy.io import fits

import gala.coordinates as gc
import gala.potential as gp
import gala.dynamics as gd

In [2]:
plt.style.use('notebook')

Here we'll see how to automatically generate data for a latex table. One of the tables we want to include in the paper are the present-day positions and velocities of the simulated dark matter particles. Let's first read the table:

In [3]:
tdm = Table(fits.getdata('../data/GSE_DM.fits'))

In [9]:
tdm

X_gal,Y_gal,Z_gal,Vx_gal,Vy_gal,Vz_gal
float32,float32,float32,float32,float32,float32
-641.7788,141.49406,-183.97186,-65.511116,-8.441251,48.682896
47.67679,22.27179,84.15445,69.719215,-61.422874,3.4637668
252.20497,-118.0045,133.71234,66.55214,42.793537,-40.005566
-89.37547,48.454453,-100.51599,-162.35555,-9.586988,-58.69164
-923.2537,311.52368,-207.74084,-52.966892,32.623882,-84.48949
-20.237411,19.343277,-29.004517,-102.44625,71.465645,-40.213554
235.41873,42.179565,3.0851746,-31.852486,78.42692,-69.01593
-218.46303,742.0701,-152.4661,7.211487,118.12472,-11.685992
5.477768,7.0532074,-1.3725739,-247.22664,-12.10263,-118.69226


If we were to construct the table by hand, it would look something like this:

\begin{deluxetable}{c c c c c c}
\tablehead{
$X$ & $Y$ & $Z$ & $V_X$ & $V_Y$ & $V_Z$ \\
kpc & kpc & kpc & $\rm km\,s^{-1}$ & $\rm km\,s^{-1}$ & $\rm km\,s^{-1}$
}
\decimals
\setlength{\tabcolsep}{3pt}
\startdata
-641.8 & 141.5 & -184.0 & -65.5 & -8.4 & 48.7 \\ 
47.7 & 22.3 & 84.2 & 69.7 & -61.4 & 3.5 \\ 
252.2 & -118.0 & 133.7 & 66.6 & 42.8 & -40.0 \\ 
\enddata
\caption{Present-day positions and velocities of simulated dark-matter particles in Galactocentric coordinates.}
\label{table:dm}
\end{deluxetable}

This involves looking up positions of dark matter particles and typing them up manually in the table. This process can easily lead to typos and takes a lot of time. Fortunately, we can automatically generate a part of the table with data like this:

In [10]:
Nrows = 3

f = open('../data/table_dm.tex', 'w')
for i in range(Nrows):
    f.write('{:.1f} & {:.1f} & {:.1f} & {:.1f} & {:.1f} & {:.1f} \\\\ \n'.format(tdm['X_gal'][i], tdm['Y_gal'][i], 
                                                                                 tdm['Z_gal'][i], tdm['Vx_gal'][i], 
                                                                                 tdm['Vy_gal'][i], tdm['Vz_gal'][i]))
f.close()

Let's see what this snippet does line by line:

`Nrows = 3` -- set how many lines in the table we want

`f = open('../data/table_dm.tex', 'w')` -- create a new file (or overwrite an existing file) table_dm.tex for writing in our data directory (you need to change this path to your system)

`for i in range(Nrows):` -- what we'll do next is cycle through the first Nrows lines in the table and print its contents to the table. Since we are repeating the same action for all the lines, we'll use python's for command Nrows times

`f.write('{:.1f} & ... \\\\ \n'.format(tdm['X_gal'][i], ...` (snippet) -- this command actually writes to file everything between the quotation marks. So, we are constructing a string with 6 entries, one for each position / velocity component. 

In the snippet, we only have the first column, the X positions stored in column `tdm['X_gal']`. To have python print the first element of this column, we'd use this: `'{:}'.format(tdm['X_gal'][0])`. Because we want this to work for any element, not just the first one, we replace the index 0 with a changing index `i`, which is determined in the for loop: `'{:}'.format(tdm['X_gal'][i])`.

Next, we can also tell python how to format this number by adding the format identifier after the colon: `'{:.1f}'` -- this tells python the number is a float `f` (that is a decimal point number), and to only keep one digit after the decimal point `.1`.

Different entries are separated by `&`, as in any latex table. After the last entry we have `\\\\`, which will tell latex to start a new line. Compared to writing a table manually, these are 4 backslashes, instead of just 2. This is because a single backslash is a special escape character in python, so we need to tell it to actually print it by adding one additional backslash for every that we need. Then the line that actually prints out will have 2.

`\n` tells python to start a new line in the file it's writing.

`f.close()` -- close the file

A file `table_dm.tex` should appear in the data directory. First upload it to overleaf, and then you can write the table like this:

\begin{deluxetable}{c c c c c c}
\tablehead{
$X$ & $Y$ & $Z$ & $V_X$ & $V_Y$ & $V_Z$ \\
kpc & kpc & kpc & $\rm km\,s^{-1}$ & $\rm km\,s^{-1}$ & $\rm km\,s^{-1}$
}
\decimals
\setlength{\tabcolsep}{3pt}
\startdata
\input{table_dm.tex}
\enddata
\caption{Present-day positions and velocities of simulated dark-matter particles in Galactocentric coordinates.}
\label{table:dm}
\end{deluxetable}

The structure of the table is the same, but instead of us looking up the numbers and typing them directly, we now just call the `input` command and insert the data we just printed out with python.